In [99]:
import pandas as pd
from datetime import datetime
import numpy as np
import glob
import os
import xlrd
import re
import openpyxl

## Clean up overdose data

In [100]:
orig_cols_to_keep = ["REPORTER_DEA_NO", "REPORTER_STATE", "REPORTER_ZIP", "REPORTER_COUNTY", "DRUG_CODE", "TRANSACTION_CODE", "DRUG_NAME", "QUANTITY", "TRANSACTION_DATE", "CALC_BASE_WT_IN_GM", "DOSAGE_UNIT",
                "TRANSACTION_ID", "Product_Name", "Ingredient_Name", "Measure", "dos_str"]
# keeping more columns than is probably necessary
# just want to make sure we have everything we need, as well as have the opportunity to catch any further filtering we miss at first
pd.set_option('display.max_columns', None)


In [101]:
def find_year(TRANSACTION_DATE):
    """
    Args:
        TRANSACTION_DATE (str): date in format MMDDYYYY

    Returns:
        int: year
    """
    TRANSACTION_DATE = str(TRANSACTION_DATE)
    
    return int(TRANSACTION_DATE[-4:])

# quick test 
print(f"testing the find_year function: {find_year(12202019)}")


def find_month(TRANSACTION_DATE):
    """
    Args:
        TRANSACTION_DATE (str): date in format MMDDYYYY

    Returns:
        int: month
    """
    TRANSACTION_DATE = str(TRANSACTION_DATE)
    
    return int(TRANSACTION_DATE[:2])

# quick test 
print(f"testing the find_month function: {find_month(12202019)}")

testing the find_year function: 2019
testing the find_month function: 12


In [102]:
# to load in the data, we need to truncate the amount of columns we use as well as the states
cols_to_keep = ["REPORTER_DEA_NO", "REPORTER_STATE", "REPORTER_ZIP", "REPORTER_COUNTY", "DRUG_CODE", "TRANSACTION_CODE", "DRUG_NAME", "QUANTITY", "TRANSACTION_DATE", "Product_Name"]

# we know we need Florida, Texas, and Washington
states = ["FL", "TX", "WA"]
# since we are normalizing based on population, I think we should pick states that are regionally close to our target states
# we can change this later as a group, but I have these selected below:

# Florida comparison states: Georgia, Alabama, Missisippi, South Carolina, Tennessee
fl_states = ["GA", "AL", "MS", "SC", "TN"]

# Texas comparison states: Oklahoma, Louisiana, New Mexico, Arkansas, Kansas
tx_states = ["OK", "LA", "NM", "AR", "KS"]

# Washington comparison states: Oregon, Idaho, Montana, Nevada, Wyoming
wa_states = ["OR", "ID", "MT", "NV", "WY"]

# create list of all states to use
variable_states = []
variable_states.extend(fl_states)
variable_states.extend(tx_states)
variable_states.extend(wa_states)

# append variable states to our original list
states.extend(variable_states)


In [103]:
# now, load in our data as an iterator so we can load in chunks
it = pd.read_csv("00_source_data/arcos_all_washpost.tsv", chunksize=500_000, sep='\t', usecols = cols_to_keep) # may have to change chunksize depending on your computer's memory

# init empty dataframe
df = pd.DataFrame()

for chunk in it:
    # extract year out of date column
    chunk["year"] = chunk["TRANSACTION_DATE"].apply(lambda x: find_year(x))
    chunk["month"] = chunk["TRANSACTION_DATE"].apply(lambda x: find_month(x))

    # ensure we're working in the correct date range
    filtered_chunk = chunk[chunk["year"] > 2002]
    filtered_chunk = filtered_chunk[filtered_chunk["year"] < 2016]

    # filter out the states we want
    filtered_chunk = filtered_chunk[filtered_chunk["REPORTER_STATE"].isin(states)]

    df = df.append(filtered_chunk)
    break
df

df_overdoses = df.copy() # keep a copy of this df for later filtering

In [104]:
df_overdoses.head()

,REPORTER_DEA_NO,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,TRANSACTION_CODE,DRUG_CODE,DRUG_NAME,QUANTITY,TRANSACTION_DATE,Product_Name,year,month
6224,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,2.0,11232009,HYDROCODONE BIT/IBUPROFEN 7.5MG/200M,2009,11
6225,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,2.0,12172012,HYDROCODONE BIT./ACETAMINOPHEN TABS.,2012,12
6226,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,1.0,2192007,HYDROCODONE/IBUPROFEN 7.5MG/200MG TA,2007,21
6227,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,1.0,6102011,HYDROCODONE BIT/ACETA 10MG/325MG USP,2011,61
6228,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,1.0,6022008,HYDROCODONE BIT/ACETA 7.5MG/325MG US,2008,60


In [105]:
# now that we have our columns and states filtered, let's export this as a csv and store the (much smaller) result on github

df_overdoses.to_csv("05_cleaned_data/arcos_all_washpost_clean.csv", index=False)

## Clean up cause of death data

In [106]:
path = r'00_source_data/cause_of_death' # point to correct folder
filenames = glob.glob(path + "/*.txt")

df = pd.DataFrame()

for f in filenames:
    temp = pd.read_csv(f, index_col=None, header=0, sep='\t')
    # we're getting some extraneous notes at the bottom - let's just drop for now
    temp.dropna(subset={'County'}, inplace=True)
    
    df = pd.concat([df, temp], axis=0, ignore_index=True)

In [107]:
df

,Notes,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths
0,NaN,"Autauga County, AL",1001.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,397.0
1,NaN,"Baldwin County, AL",1003.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0
2,NaN,"Baldwin County, AL",1003.0,2003.0,2003.0,All other alcohol-induced causes,A9,14.0
3,NaN,"Baldwin County, AL",1003.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,1479.0
4,NaN,"Barbour County, AL",1005.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,287.0
...,...,...,...,...,...,...,...,...
57236,NaN,"Sweetwater County, WY",56037.0,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,251
57237,NaN,"Teton County, WY",56039.0,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,95
57238,NaN,"Uinta County, WY",56041.0,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,142
57239,NaN,"Washakie County, WY",56043.0,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,81


In [108]:
# helper functions to separate county and state

def abtract_state(county):
    """
    Args:
        county (str): county name

    Returns:
        str: state
    """
    return county.split(", ")[1]



def abstract_county(county):
    """
    Args:
        county (str): county name

    Returns:
        str: county
    """
    return county.split(", ")[0]


df["State"] = df.apply(lambda x: abtract_state(x["County"]), axis=1)
df["County"] = df.apply(lambda x: abstract_county(x["County"]), axis=1)

df.drop(columns={"Notes"}, inplace=True)

df_cause_of_death = df.copy() # keep a copy of this df for later filtering

In [109]:
df_cause_of_death.head()

,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,State
0,Autauga County,1001.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,397.0,AL
1,Baldwin County,1003.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0,AL
2,Baldwin County,1003.0,2003.0,2003.0,All other alcohol-induced causes,A9,14.0,AL
3,Baldwin County,1003.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,1479.0,AL
4,Barbour County,1005.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,287.0,AL


In [110]:
df_cause_of_death.to_csv("05_cleaned_data/cause_of_death_clean.csv", index=False)

## Adding in County Population data

[Census county pop. data, 2000-2010](https://www.census.gov/data/tables/time-series/demo/popest/intercensal-2000-2010-counties.html)<br>
[Census county pop. data, 2010-2019](https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html)<br>
For both, just select the appropriate states on the webpage. We will clean and merge as needed in this notebook.



#### Guide to cleaning - 2000s data

The way the 2000s excel files are formatted, we can clean the data in the following way

- load in with header=3
- drop null on any of the populations
    - notes at the bottom will be removed
- drop unnamed 1, 12, and 13
    - these contain redundant data about populations from specific dates
    - Unnamed 12 is 2010s pop - will be redundant as our next dataset has this as well. Using the newer data
- drop first row
    - state as a whole
- rename Unnamed: 0 to county


In [131]:
pops00 = pd.DataFrame()

# add every excel file in 00_source_data/county_pop/2000s to pops00

path = r"00_source_data/county_pop/2000s/" # point to correct folder
filenames = glob.glob(path + "*.xls")

for f in filenames:

    # read in current file with header = 3
    temp = pd.read_excel(f, header = 3)

    # regex to pull out state from filename
    r = re.search("(2000s)(.)(\w+)", f)[3]
    temp["State"] = r[:2].upper()
    
    # drop null on any of the years
    temp.dropna(subset=[2000], inplace=True)

    #drop useless columns
    temp.drop(columns={"Unnamed: 1", "Unnamed: 12", "Unnamed: 13"}, inplace=True)

    # drop first row
    temp = temp.iloc[1:, :]

    # rename some cols
    temp.rename(columns={"Unnamed: 0": "County"}, inplace=True)

    # remove period at beginning of each county
    temp["County"] = temp["County"].apply(lambda x: x[1:])

    pops00 = pd.concat([pops00, temp], axis=0, ignore_index=True)

# quick peek at the data
pops00.head()


,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,State
0,Arkansas County,20776.0,20593.0,20351.0,19981.0,19967.0,19954.0,19785.0,19434.0,19275.0,19134.0,AK
1,Ashley County,24179.0,23824.0,23745.0,23469.0,23156.0,22770.0,22501.0,22229.0,22148.0,21920.0,AK
2,Baxter County,38446.0,38331.0,38506.0,38772.0,39334.0,39992.0,40867.0,41491.0,41519.0,41561.0,AK
3,Benton County,154744.0,159258.0,164926.0,171742.0,179010.0,186756.0,195569.0,203664.0,210807.0,216620.0,AK
4,Boone County,34051.0,34258.0,34448.0,34554.0,34900.0,35492.0,36056.0,36759.0,36974.0,36844.0,AK


#### Guide to cleaning - 2010s data

The way the 2010s excel files are formatted, we can clean the data in the following way

- load in with header=3
- drop null on any of the populations
    - notes at the bottom will be removed
- drop census, estimates base
- drop first row
    - state as a whole
- rename Unnamed: 0 to county


In [132]:
pops10 = pd.DataFrame()

# add every excel file in 00_source_data/county_pop/2000s to pops00

path = r"00_source_data/county_pop/2010s" # point to correct folder
filenames = glob.glob(path + "/*.xlsx")

for f in filenames:

    # read in current file with header = 3
    temp = pd.read_excel(f, header = 3)

    # regex to pull out state from filename
    r = re.search("(2010s)(.)(\w+)", f)[3]
    temp["State"] = r[:2].upper()
    
    # drop null on any of the years
    temp.dropna(subset=[2010], inplace=True)

    #drop useless columns
    temp.drop(columns={"Census", "Estimates Base"}, inplace=True)

    # drop first row
    temp = temp.iloc[1:, :]

    # rename some cols
    temp.rename(columns={"Unnamed: 0": "County"}, inplace=True)

    # remove period at beginning of each county
    temp["County"] = temp["County"].apply(lambda x: x[1:])

    # strip state from county
    temp["County"] = temp["County"].apply(lambda x: x.split(", ")[0])

    pops10 = pd.concat([pops10, temp], axis=0, ignore_index=True)

# quick peek at the data
pops10.head()


,County,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,State
0,Arkansas County,19009.0,18871.0,18964.0,18755.0,18479.0,18330.0,18154.0,17872.0,17726.0,17486.0,AK
1,Ashley County,21829.0,21673.0,21515.0,21278.0,20956.0,20838.0,20530.0,20311.0,20012.0,19657.0,AK
2,Baxter County,41510.0,41349.0,41080.0,40983.0,40888.0,41138.0,41131.0,41308.0,41624.0,41932.0,AK
3,Benton County,222593.0,229171.0,234938.0,239461.0,244841.0,251591.0,259212.0,266585.0,272266.0,279141.0,AK
4,Boone County,36893.0,37086.0,37333.0,37328.0,37116.0,37119.0,37259.0,37459.0,37385.0,37432.0,AK


In [133]:
# melt both dfs to get tidy format

pops00 = pops00.melt(["County", "State"])
pops10 = pops10.melt(["County", "State"])

# rename columns accordingly
pops00.rename(columns={"variable": "Year", "value": "Population"}, inplace=True)
pops10.rename(columns={"variable": "Year", "value": "Population"}, inplace=True)

# concatenate the two dfs
pops = pd.concat([pops00, pops10])


In [134]:
# check that we have the same number of counties between datasets
assert len(pops00["County"].unique()) == len(pops10["County"].unique())

# check that we have the same number of counties every year
# first, create a df with the number of counties per year
pops_county_check = pops.groupby(["State", "Year"])["County"].count().reset_index()
#pops10_county_check = pops10.groupby(["State", "Year"])["County"].count().reset_index()

In [135]:
# group the sum of counties by year and state - will help us check if number of counties changes over the years
grouped_states = pops_county_check.groupby(["Year", "State"])["County"].sum().reset_index()
#grouped_states10 = pops10_county_check.groupby(["Year", "State"])["County"].sum().reset_index()

In [136]:
pops[pops["State"] == "AK"].Year.value_counts()

2000    75
2001    75
2018    75
2017    75
2016    75
2015    75
2014    75
2013    75
2012    75
2011    75
2010    75
2009    75
2008    75
2007    75
2006    75
2005    75
2004    75
2003    75
2002    75
2019    75
Name: Year, dtype: int64

In [119]:
# using the above query, we should be able to assert that the number of counties per year is the same
# below statement should always equal zero

assert (grouped_states.duplicated(subset=["Year", "State"]).sum() == 0)
#assert (grouped_states10.duplicated(subset=["Year", "State"]).sum() == 0)


# ensure no duplicate values
assert pops.duplicated().sum() == 0

## Final 3 datasets

We should have: (UNSURE IF WE SHOULD EXTEND DATE RANGES, CURRENTLY 3 YEARS BEFORE AND AFTER POLICY IMPLEMENTATION)

- Florida and Georgia 2007 - 2013
- Texas and Oklahoma 2004 - 2010
- Washington and Oregon 2009 - 2015

### Drug overdose - broken down by state

In [120]:
# Florida and Georgia

overdoses_fl = df_overdoses.copy()
overdoses_tx = df_overdoses.copy()
overdoses_wa = df_overdoses.copy()

overdoses_fl = overdoses_fl[(overdoses_fl["REPORTER_STATE"] == "FL") | (overdoses_fl["REPORTER_STATE"].isin(fl_states))]
overdoses_tx = overdoses_tx[(overdoses_tx["REPORTER_STATE"] == "TX") | (overdoses_tx["REPORTER_STATE"]).isin(tx_states)]
overdoses_wa = overdoses_wa[(overdoses_wa["REPORTER_STATE"] == "WA") | (overdoses_wa["REPORTER_STATE"]).isin(wa_states)]



# filter appropriate years
fl_start = 2007
fl_end = 2013

tx_start = 2004
tx_end = 2010

wa_start = 2009
wa_end = 2015


overdoses_fl = overdoses_fl[(overdoses_fl["year"] >= fl_start) & (overdoses_fl["year"] <= fl_end)]
overdoses_tx = overdoses_tx[(overdoses_tx["year"] >= tx_start) & (overdoses_tx["year"] <= tx_end)]
overdoses_wa = overdoses_wa[(overdoses_wa["year"] >= wa_start) & (overdoses_wa["year"] <= wa_end)]


### Cause of death - broken down by state

In [121]:
# ensure states are now in their full form to match the cause of death data
fl_states = ["Georgia", "Alabama", "Missisippi", "South Carolina", "Tennessee"]

tx_states = ["Oklahoma", "Louisiana", "New Mexico", "Arkansas", "Kansas"]

wa_states = ["Oregon", "Idaho", "Montana", "Nevada", "Wyoming"]

In [122]:
deaths_fl = df_cause_of_death.copy()
deaths_tx = df_cause_of_death.copy()
deaths_wa = df_cause_of_death.copy()

deaths_fl = deaths_fl[(deaths_fl["State"] == "Florida") | (deaths_fl["State"].isin(fl_states))]
deaths_tx = deaths_tx[(deaths_tx["State"] == "Texas") | (deaths_tx["State"].isin(tx_states))]
deaths_wa = deaths_wa[(deaths_wa["State"] == "Washington") | (deaths_wa["State"].isin(wa_states))]

deaths_fl = deaths_fl[(deaths_fl["Year"] >= fl_start) & (deaths_fl["Year"] <= fl_end)]
deaths_tx = deaths_tx[(deaths_tx["Year"] >= tx_start) & (deaths_tx["Year"] <= tx_end)]  
deaths_wa = deaths_wa[(deaths_wa["Year"] >= wa_start) & (deaths_wa["Year"] <= wa_end)]

### export all to csv

In [123]:
overdoses_fl.to_csv("05_cleaned_data/overdoses_fl.csv", index=False)
overdoses_tx.to_csv("05_cleaned_data/overdoses_tx.csv", index=False)
overdoses_wa.to_csv("05_cleaned_data/overdoses_wa.csv", index=False)

deaths_fl.to_csv("05_cleaned_data/deaths_fl.csv", index=False)
deaths_tx.to_csv("05_cleaned_data/deaths_tx.csv", index=False)
deaths_wa.to_csv("05_cleaned_data/deaths_wa.csv", index=False)

## Notes for the group

- may need to filter out a couple more columns - haven't done this yet as I don't want to accidentally delete something we need
- overdose data is only broken down by year unless i messed something up - overdose analysis will have to be less granular